In [1]:
import re
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:

# Function to parse the log file
def parse_log_file(log_file_path):
    iterations = []
    mious = []
    maccs = []
    learning_rates = []

    last_iter = None
    last_lr = None

    with open(log_file_path, 'r') as f:
        for line in f:
            # Parse training iteration and learning rate
            train_match = re.search(r"Iter\(train\)\s*\[\s*(\d+)/\d+\]\s*lr:\s*([0-9\.e+-]+)", line)
            if train_match:
                last_iter = int(train_match.group(1))
                last_lr = float(train_match.group(2))
                continue

            # Parse validation metrics (mIoU and mAcc)
            val_match = re.search(r"mIoU:\s*(\d+\.\d+)\s*mAcc:\s*(\d+\.\d+)", line)
            if val_match:
                if last_iter is not None:
                    iterations.append(last_iter)
                    mious.append(float(val_match.group(1)))
                    maccs.append(float(val_match.group(2)))
                    learning_rates.append(last_lr)

    return iterations, mious, maccs, learning_rates

# New function to plot interactively with Plotly
def plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, x_range):
    # Create a figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add Top-1 Accuracy trace
    fig.add_trace(
        go.Scatter(x=epochs, y=mAcc, name='Mean Accuracy', line=dict(color='blue')),
        secondary_y=False,
    )

    # Add Top-5 Accuracy trace
    fig.add_trace(
        go.Scatter(x=epochs, y=mIoU, name='Mean IoU', line=dict(color='orange')),
        secondary_y=False,
    )

    # Add Learning Rate trace on secondary y-axis
    fig.add_trace(
        go.Scatter(x=epochs, y=lrs, name='Learning Rate', line=dict(color='green')),
        secondary_y=True,
    )

    # Update layout for interactivity and axes
    fig.update_layout(
        title='Validation Performance and Training Learning Rate vs Epoch',
        xaxis_title='Epoch',
        yaxis_title='Performance (%)',
        yaxis2_title='Learning Rate',
        hovermode='x unified',  # Vertical crosshair on x-hover, shows values for all lines
        legend=dict(yanchor='top', y=1.15, xanchor='right', x=1.15),  # Top-right legend
    )

    # Set x-axis ranges
    fig.update_xaxes(range=x_range)
    
    # Set y-axis ranges
    fig.update_yaxes(range=[0, 100], dtick=10, secondary_y=False)
    fig.update_yaxes(secondary_y=True)  # Auto-range for LR

    # Show the figure (interactive in Jupyter)
    fig.show()


In [3]:
adam_range = [0, 310]
sgd_range = [0, 120000]

In [4]:
# Ablation 01 - SEBNet BaselineHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy - 72.72 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb6_cityscapes_ohem/20250822_162921/20250822_162921.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [5]:
# Ablation 01 - SEBNet BaselineHead SGD, Scratch, Batch Size 6, OHEM Cross Entropy - 72.43 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb6_cityscapes_ohem_scratch/20250822_162814/20250822_162814.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [6]:
# Ablation 01 - SEBNet BaselineHead SGD, Pretrained, Batch Size 8, OHEM Cross Entropy - 72.34 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb8_cityscapes_ohem/20250822_163056/20250822_163056.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,120000])

In [7]:
# Ablation 01 - SEBNet BaselineHead SGD, Scratch, Batch Size 8, OHEM Cross Entropy - 72.7 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb8_cityscapes_ohem_scratch/20250822_163019/20250822_163019.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,120000])

In [8]:
# Ablation 02, SEBNet BaselinePHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy - 78.98 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-head_1xb6_cityscapes/20250825_134022/20250825_134022.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [9]:
# Ablation 03, SEBNet BaselineDHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 4, Edge Loss Weight 20.0 - 73.90 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-d-head_1xb6_cityscapes/20250825_134116/20250825_134116.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [10]:
# Ablation 19, SEBNet BaselinePDHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy - 79.57 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-head_1xb6_cityscapes/20250825_134133/20250825_134133.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [11]:
# Ablation 25, SEBNet BaselinePDBASHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy - 78.87 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-bas-head_1xb6_cityscapes/20250825_134156/20250825_134156.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [12]:
# Ablation 03, SEBNet BaselineDHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - 73.17 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-d-head_1xb6_cityscapes/20250826_135921/20250826_135921.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [13]:
# Ablation 09 - SEBNet BaselineCASENetHeadEarlierLayers SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - 73.45 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-casenet-head-earlier-layers_1xb6_cityscapes/20250827_123728/20250827_123728.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [14]:
# Ablation 12 - SEBNet BaselineDMultiLabelHeadEarlierLayers SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - 74.40 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-d-multilabel-head-earlier-layers_1xb6_cityscapes/20250827_123750/20250827_123750.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [15]:
# Ablation 10 - SEBNet BaselineDFFHeadEarlierLayers SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - 73.69 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-dff-head-earlier-layers_1xb6_cityscapes/20250827_123829/20250827_123829.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [16]:
# Ablation 08 - SEBNet BaselineDHeadEarlierLayers SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - 73.51 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-d-head-earlier-layers_1xb6_cityscapes/20250827_130234/20250827_130234.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [17]:
# SEBNet BaselineDSBDHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - XX.XX Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-d-sbd-head_1xb6_cityscapes/20250903_135626/20250903_135626.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [18]:
# SEBNet BaselinePDSBDHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - XX.XX Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-sbd-head_1xb6_cityscapes/20250903_135655/20250903_135655.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [19]:
# SEBNet BaselinePDSBDBASHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - XX.XX Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-sbd-bas-head_1xb6_cityscapes/20250903_135710/20250903_135710.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [20]:
# SEBNet BaselinePDSBDBASWithSBDHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - XX.XX Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-sbd-basWithSBD-head_1xb6_cityscapes/20250903_135729/20250903_135729.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [21]:
# Ablation 25 - SEBNet PIDNet SGD, Pretrained, Batch Size 6, OHEM Cross Entropy - 80.00 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-bas-head_1xb6_cityscapes/20250823_235000/20250823_235000.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [22]:
# Ablation 20 - SEBNet BaselineDSBDHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - 74.65 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-d-sbd-head_1xb6_cityscapes/20250903_225043/20250903_225043.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [23]:
# Ablation 26 - SEBNet BaselinePDSBDHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - 79.63 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-sbd-head_1xb6_cityscapes/20250903_225120/20250903_225120.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [24]:
# Ablation 27 - SEBNet BaselinePDSBDBASHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - 80.44 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-sbd-bas-head_1xb6_cityscapes/20250903_225204/20250903_225204.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [25]:
# SEBNet BaselinePDSBDBASWithSBDHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy, Edge Width 2, Edge Loss Weight 5.0 - 80.09 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-sbd-basWithSBD-head_1xb6_cityscapes/20250903_225426/20250903_225426.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [26]:
# PIDNet SGD, Pretrained, Batch Size 6, OHEM Cross Entropy - 80.00 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/pidnet-l_2xb6-120k_1024x1024-cityscapes/20250906_103110/20250906_103110.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,240064])

In [27]:
# Ablation 25 - SEBNet BaselinePDBASHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy - 79.99 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-bas-head_1xb6_cityscapes/20250911_103625/20250911_103625.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [28]:
# Ablation 25 - SEBNet BaselinePDBASHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy - 80.47 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-bas-head_1xb6_cityscapes/20250906_105242/20250906_105242.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,240064])

In [29]:
# Ablation 24 - SEBNet BagBaselinePSBDBASHead SGD, Pretrained, Batch Size 6, OHEM Cross Entropy - 80.23 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-sbd-bas-head-fused_1xb6_cityscapes/20250906_103054/20250906_103054.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,240064])

In [30]:
# Ablation 28 - SEBNet BaselinePDSBDBASHead SGD, Mapillary, 480K Iters - 80.23 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-sbd-bas-head_1xb6_mapillaryv2/20250909_160646/20250909_160646.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,480000])

In [31]:
# Ablation 28 - SEBNet ConditionedBaselinePSBDBASHead SGD, Mapillary, 480K Iters - 80.23 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-sbd-bas-head-conditioned_1xb6_mapillaryv2/20250909_160714/20250909_160714.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,480000])